In [ ]:
import pandas as pd
import numpy as np
import ast
import json
import os
import re

In [ ]:
%%time
lyrics_df = pd.read_csv('res_all.csv', index_col=0)
lyrics_df.drop('album_url', axis=1, inplace=True)
lyrics_df.drop_duplicates(subset='text', keep='last', inplace=True)
lyrics_df = lyrics_df[lyrics_df['text'].apply(len) > 300]
lyrics_df = lyrics_df[['artist', 'album', 'song', 'featuring', 'text']]
lyrics_df.columns = ['Author', 'Album', 'Song', 'Featuring', 'Lyrics']
lyrics_df = lyrics_df[lyrics_df['Song'].apply(lambda x: '(Skit)' not in x and '(Live' not in x)]
lyrics_df.sort_values(by=['Author', 'Album'], inplace=True)
lyrics_df.reset_index(drop=True, inplace=True)
print(lyrics_df.shape)

In [ ]:
lyrics_df.head()

In [ ]:
# with open("singers_aliases.json", "rb") as file:
#     rename_dict = json.load(file)
    
# lyrics_df['Author'] = lyrics_df['Author'].apply(lambda author: rename_dict.get(author, author))

In [ ]:
lyrics_df.to_csv('songs_dataset.csv')

# Experiments

In [ ]:
def get_part_lyrics(part):
    part = part[1:]
    return '\n'.join(part)

In [ ]:
def get_part_info(part, all_singers, author, album, song):
    singer = None
    brackets = part[0]
    brackets_parts = brackets[1:-1].split(':')
    if len(brackets_parts) == 1:
        singer = all_singers[0]
        part_name = brackets_parts[0].strip()
    else:
        part_singers = brackets_parts[1]
        part_singers = list(map(str.strip, re.split(',|&', part_singers)))
        for part_singer in part_singers:
            if part_singer in all_singers:
                singer = part_singer
                break
        part_name = brackets_parts[0]
    part_lyrics = get_part_lyrics(part)
    if part_lyrics == '' or not singer:
        return None
    return [author, album, song, singer, part_name, part_lyrics]

In [ ]:
song_pathes = ['Kanye West_NBA 2K13 Soundtrack_Mercy_2 Chainz_Pusha T_Big Sean.txt',
               'Logic_Confessions of a Dangerous Mind_Homicide_Eminem.txt',
               'Eminem_Kamikaze_Venom.txt',
               'Lil Pump_Lil Pump_Gucci Gang.txt']

songs_info = []

for song_path in song_pathes:
    song_text = open(f'lyrics_examples/{song_path}').read()
    author, album, song, *feats = os.path.splitext(song_path)[0].split('_')
    all_singers = [author] + feats
    
    indices = [i for i, ltr in enumerate(song_text) if ltr == '[' and song_text[i-1] == '\n'] + [len(song_text)]
    song_parts = [song_text[indices[i]:indices[i+1]].strip().split('\n') for i, index in enumerate(indices[:-1])]
    song_parts_info = [get_part_info(song_part, all_singers, author, album, song) for song_part in song_parts]
    song_parts_info = filter(None, song_parts_info)
    song_parts_info
    
    songs_info.extend(song_parts_info)

songs_info_df = pd.DataFrame(songs_info, columns=['Author', 'Album', 'Song', 'Singer', 'Song Part', 'Lyrics'])
songs_info_df